In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import WhitespaceTokenizer
from nltk.corpus import stopwords
%matplotlib inline

### Load data

In [ ]:
show_data = pd.read_csv('../input/200000-jeopardy-questions/JEOPARDY_CSV.csv')
show_data.head()

In [ ]:
show_data.nunique()

Show number has no signifacence on the value of a question

In [ ]:
show_data.drop('Show Number', axis=1, inplace=True)
show_data.rename(columns=lambda x: x.strip(), inplace=True)
show_data = show_data[~(show_data['Round'].isin(["Final Jeopardy!", "Tiebreaker"]))]

In [ ]:
show_data['Value'] = show_data['Value'].str.replace("$", '')
show_data['Value'] = show_data['Value'].str.replace(",", '')
show_data['Value'] = show_data['Value'].astype(int)
show_data.head()

### EDA

In [ ]:
show_data['Air Date'] = pd.to_datetime(show_data['Air Date'])
str(show_data['Air Date'].dt.date.min()) + ' to ' +str(show_data['Air Date'].dt.date.max())

In [ ]:
plt_df = show_data.groupby(show_data['Air Date'].dt.year).mean('Value')
plt_df.plot(xticks=plt_df.index, rot=90);

So, dates has some effect on the values; it can be seen that around 2002 average value increased suddenly.<br/> Create two baskets fo dates pre-2002 and post-2002

In [ ]:
show_data['air_date_group'] = show_data['Air Date'].apply(lambda x: 'pre-2002' if x.year < 2002 else 'post-2002')
show_data.drop('Air Date', axis=1, inplace=True)
show_data.head()

***Assumption: `Answer` has no significance on the value***

In [ ]:
show_data.drop('Answer', axis=1, inplace=True)
show_data.head()

In [ ]:
plt_df = show_data.groupby('Round').mean('Value')
plt_df.plot.bar();

So, type of round has significance on value too.

In [ ]:
show_data['Category'].nunique()

There seems to be very high number of categories of questions. Also, based on the game rukles category doesn't seem to have much influence on the values. So dropping it.<br/>
***Assumption: `Category` has no significance on the value***

In [ ]:
show_data.drop('Category', axis=1, inplace=True)

In [ ]:
def binning(value):
    if value < 1000:
        return np.round(value, -2)
    elif value < 10000:
        return np.round(value, -3)
    else:
        return np.round(value, -4)

show_data['Value'] = show_data['Value'].apply(binning)

In [ ]:
tokenizer = WhitespaceTokenizer()
lemmatizer = WordNetLemmatizer()
def text_preprocessing(text):
    return " ".join([lemmatizer.lemmatize(w) for w in tokenizer.tokenize(text)])
show_data['Question'] = show_data['Question'].apply(text_preprocessing)
show_data.head()

In [ ]:
show_data.describe()

### Model creation

In [ ]:
y = show_data['Value']
X = show_data[['Round', 'air_date_group', 'Question']]
column_trans = ColumnTransformer([('Round', OneHotEncoder(dtype='int'),['Round']),
                                  ('air_date_group', OneHotEncoder(dtype='int'),['air_date_group']),
                                  ('Question', TfidfVectorizer(stop_words='english'), 'Question')],
                                remainder='drop')
X = column_trans.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
rfc = RandomForestClassifier(n_jobs=-1, n_estimators=500, max_features='auto', oob_score=True, verbose=1) 
rfc.fit(X_train, y_train)

In [ ]:
preds = CV_rfc.predict(X_test)
print(classification_report(y_test, preds))

### Problems with wysa chatbot:
1. The bot don't undertsand when I am not understanding what it is trying to say and looking for more explaination. <br/>`solution:` It need sto train more on understanding natural flow of converstaion and needs to know certain keywords like 'how', 'what' etc based on previous context.
2. The bot tries to keep a simple pattern all the time which might not be the case always.<br/>`soluition:`I have no idea how to solev this, maybe giving more training example and context might hlep.
3. The chatbot get confuesd when asked some questions which are not directly linked to the question answer session and gives some weird answers.<br/>`solution:` I think it will be benficial to train on out-of-context sample so it can detect and can come back to the topic.